# $\mu\text{DSTs}$

### Set-up

In [25]:
import os

import uproot
from uproot.reading import ReadOnlyDirectory
from uproot.behaviors.TBranch import HasBranches

import matplotlib.pyplot as plt
from matplotlib.axes import Axes

import numpy as np
import numpy.typing as npt

import pandas as pd
# import awkward_pandas as apd
# import awkward as ak

# import ipywidgets
# from ipywidgets import interact

%matplotlib inline

### Functions

In [2]:
def plot_th1f(
        data: ReadOnlyDirectory | HasBranches,
        key: str,
        axs: Axes,
        xlabel: str = '',
        ylabel: str = 'Frequency'
    ) -> tuple[npt.NDArray, npt.NDArray]:
    """\
    Plots a TH1F histogram for a given key from the MicroDST.

    Parameters
    ----------
    key: str
        The key for the TH1F histogram to be plotted.
    axs: Axes
        The target axes of the plot.
    xlabel: str
        The label for the x-axis. (Optional)
    ylabel: str
        The label for the y-axis. (Optional)

    Returns
    -------
    Axes
        The axes of the plot.

    """
    ax.cla()
    
    # Convert the TH1F to NumPy arrays...
    # Note: Linter is freaking out again :( ignore the red lines.
    values, bins = data[key].to_numpy()

    if not values.any():
        print(f'No values to plot for key \'{key}\'!')
        return values, bins

    axs.hist(values, bins=bins)

    axs.set_xlabel(xlabel)
    axs.set_ylabel(ylabel)

    axs.set_title(f'Key: \'{key}\'')

    return values, bins

### Opening files

In [3]:
data = uproot.open(path=os.environ['M_2010_R1_F_21035000'])

In [ ]:
# Note: Linter keeps freaking out...
# Pylance - Cannot access attribute "classnames" for class "HasBranches"

# .classnames() returns a list of all branches and their types.
# Possible types:
#   1. TH1F = 1d Histogram with a float per channel (with a maximum of 7 digit 
#      precision).
#   2. TH1I = 1d Histogram with an int per channel.
#   3. TTree = Tree / Columnar dataset.

names_dict = data.classnames()

names_dict

In [ ]:
type(data.classnames()['hDetector;1'])

In [6]:
classnames: list[str] = list(data.classnames().keys())
histograms: list[str] = [
    key for key in names_dict.keys() if names_dict[key].startswith('TH1')
]

In [77]:
# fig, ax = plt.subplots()
# 
# plot_th1f(data=data, key=histograms[4], axs=ax);

In [ ]:
# data['s;1'].show()

In [52]:
# data['mc;1'].show()  # -> Confusingly 's;1' also contains 'mc;1' data?

### Making plots

In [10]:
data['s;1']['s/energy'].arrays(library='np')['energy'];

In [27]:
df = data['s;1'].arrays(library='pd')

In [ ]:
_, ax = plt.subplots()

ax.hist(
    df['energy'],
    bins=np.linspace(0, 20, 36 + 1),
    alpha=0.5,
    label='Energy'
)

ax.hist(
    df['energyCC'],
    bins=np.linspace(0, 20, 36 + 1),
    alpha=0.5,
    label='EnergyCC'
)

ax.hist(
    df['energyNC'],
    bins=np.linspace(0, 20, 36 + 1),
    alpha=0.5,
    label='EnergyNC'
)

ax.hist(
    df['energyRM'],
    bins=np.linspace(0, 20, 50 + 1),
    alpha=0.5,
    label='EnergyRM'
)

ax.hist(
    df['energyMC'],
    bins=np.linspace(0, 20, 50 + 1),
    alpha=0.5,
    label='EnergyMC'
)

ax.set_xlabel('Energy [GeV]')
ax.set_ylabel('Interactions / GeV')

ax.set_title('s;1 (No cuts)')

ax.legend();

In [ ]:
_, ax = plt.subplots()

cut_df = df[df['isInFidVolCC'] & df['isGoodDataQuality']]

ax.hist(
    cut_df['energyMC'],
    bins=np.linspace(0, 20, 36 + 1),
    alpha=0.5,
    label='MC Energy'
)

ax.hist(
    cut_df['energy'],
    bins=np.linspace(0, 20, 36 + 1),
    alpha=0.5,
    label='Energy'
)

ax.set_xlabel('Energy [GeV]')
ax.set_ylabel('Interactions / GeV')

ax.set_title('s;1 (Quality + containment cuts)')

ax.legend();

In [ ]:
_, ax = plt.subplots()

cut_df = df[df['isInFidVolCC'] & df['isGoodDataQuality']]

est_err = (cut_df['energy'] - cut_df['energyMC']) / cut_df['energyMC']

ax.hist(
    est_err,
    bins=np.linspace(-1, 1, 36 + 1),
    alpha=0.5
)

ax.set_xlabel('Energy [GeV]')
ax.set_ylabel('Interactions / GeV')

ax.set_title('s;1 (Quality + containment cuts) - Energy Estimation Errors');

In [ ]:
_, ax = plt.subplots()

cut_df = df[~df['isInFidVolCC'] & df['isGoodDataQuality']]

ax.hist(
    cut_df['energyMC'],
    bins=np.linspace(0, 20, 36 + 1),
    alpha=0.5,
    label='MC Energy'
)

ax.hist(
    cut_df['energy'],
    bins=np.linspace(0, 20, 36 + 1),
    alpha=0.5,
    label='Energy'
)

ax.set_xlabel('Energy [GeV]')
ax.set_ylabel('Interactions / GeV')

ax.set_title('s;1 (Quality + non-contained cuts)')

ax.legend();

As expected, energy is underestimated for poorly contained events.

In [ ]:
_, ax = plt.subplots()

cut_df = df[~df['isInFidVolCC'] & df['isGoodDataQuality']]

est_err = (cut_df['energy'] - cut_df['energyMC']) / cut_df['energyMC']

ax.hist(
    est_err,
    bins=np.linspace(-1, 1, 36 + 1),
    alpha=0.5
)

ax.set_xlabel('Energy [GeV]')
ax.set_ylabel('Interactions / GeV')

ax.set_title('s;1 (Quality + bon-contained cuts) - Energy Estimation Errors');